In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchain_community -q
!pip install pdfplumber -q
!pip install chromadb -q
!pip install transformers peft accelerate bitsandbytes -q
!pip install  sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   

In [3]:
path_ins = '/content/drive/MyDrive/datafile/kb운전자보험.pdf'
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader(path_ins)
data = loader.load()

In [4]:
data[:5]

[Document(metadata={'source': '/content/drive/MyDrive/datafile/kb운전자보험.pdf', 'file_path': '/content/drive/MyDrive/datafile/kb운전자보험.pdf', 'page': 0, 'total_pages': 168, 'Author': 'kbins', 'Creator': 'Hwp 2018 10.0.0.8223', 'Producer': 'Hancom PDF 1.3.0.534', 'CreationDate': "D:20240422094713+09'00'", 'ModDate': "D:20240422094713+09'00'", 'PDFVersion': '1.4'}, page_content='KB스마트운전자보험\n'),
 Document(metadata={'source': '/content/drive/MyDrive/datafile/kb운전자보험.pdf', 'file_path': '/content/drive/MyDrive/datafile/kb운전자보험.pdf', 'page': 1, 'total_pages': 168, 'Author': 'kbins', 'Creator': 'Hwp 2018 10.0.0.8223', 'Producer': 'Hancom PDF 1.3.0.534', 'CreationDate': "D:20240422094713+09'00'", 'ModDate': "D:20240422094713+09'00'", 'PDFVersion': '1.4'}, page_content='목 차\n가입자 유의사항\n주요내용 요약서\n보험용어 해설\nKB스마트운전자보험 보통약관\n제1관 목적 및 용어의 정의·························································································································1\n제1조(목적) ····································

In [5]:
pages = []
for row in data:
    if isinstance(row.page_content, str):
        pages.append(row.page_content)
pages[:5]

['KB스마트운전자보험\n',
 '목 차\n가입자 유의사항\n주요내용 요약서\n보험용어 해설\nKB스마트운전자보험 보통약관\n제1관 목적 및 용어의 정의·························································································································1\n제1조(목적) ··································································································································1\n제2조(용어의 정의)······················································································································1\n제2관 보험금의 지급·····································································································································2\n제3조(보험금의 지급사유)··········································································································2\n제4조(보험금 지급에 관한 세부규정) ····················································································2\n제5조(보험금을 지급하지 않는 사유) ····················································································3\n제6조(보험금 지급사유의 통지) ··············

In [6]:
import chromadb
from sentence_transformers import SentenceTransformer

# Chroma 클라이언트 생성
client = chromadb.Client()
collection = client.get_or_create_collection(name="insurance_documents")

# SentenceTransformer 모델 로드
model_name = 'jhgan/ko-sroberta-multitask' # 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embedding_model = SentenceTransformer(model_name)

# 텍스트를 문장 단위로 분리하고 벡터화
sentences = pages  # 줄바꿈 기준으로 분리
embeddings = embedding_model.encode(sentences)

# Chroma에 문서 추가
collection.add(
    documents=sentences,
    embeddings=embeddings.tolist(),
    ids=[f"doc_{i}" for i in range(len(sentences))]
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# !huggingface-cli login

In [8]:
# !pip install -U bitsandbytes

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 모델 경로 (Hugging Face 허브 또는 로컬 경로)
model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 모델 로드 (FP16 설정)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # FP16 설정
    device_map="auto"  # GPU/CPU 자동 할당
)

# 모델 준비 완료
print("모델이 성공적으로 FP16으로 로드되었습니다!")

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

모델이 성공적으로 FP16으로 로드되었습니다!


In [10]:
def query_with_rag(question, collection):
    # 질문 임베딩 생성
    question_embedding = embedding_model.encode([question])

    # Chroma에서 유사한 문서 검색
    results = collection.query(
        query_embeddings=question_embedding.tolist(),
        n_results=5  # 상위 5개 결과 반환
    )

    # 검색된 문서 내용 결합
    related_texts = "\n".join(results['documents'][0])

    # LLaMA 3.2 입력 준비 (프롬프트 생성)
    prompt = f"다음 정보를 바탕으로 질문에 대해 250자 안으로 요약하여 답하세요:\n\n{related_texts}\n\n질문 :{question}\n답변:"

    # LLaMA 3.2로 답변 생성
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=200)

    # 답변 디코딩 및 반환
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 예시 질문 실행
question = "이 보험 상품의 보장 범위는 무엇인가요?"
answer = query_with_rag(question, collection)
output = answer.split('답변:')[-1]
print(f'답변 :{output}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


답변 : 이 보험 상품은 피보험자의 신체상해를 보장하는 보험입니다. 보험금은 보험가입금액에 따라 지급되며, 보험가입금액은 보험료 납입을 통해 계산됩니다. 또한, 보험금은 보험가입금액에 따라 지급되며, 보험가입금액은 보험료 납입을 통해 계산됩니다. 보험금은 보험가입금액에 따라 지급되며, 보험가입금액은 보험료 납입을 통해 계산됩니다. 보험금은 보험가입금액에 따라 지급되며, 보험가입금액은 보험료 납입을 통해 계산됩니다. 보험금은 보험가입금액에 따라 지급되며, 보험가입금액은 보험료 납입을 통해 계산


In [11]:
# text = ':  이 보험 상품의 보장 범위는 피보험자의 상해에 대한 위험을 보장합니다. 보험금 지급사유가 발생한 때 보험수익자에게 보험금을 지급합니다. 보험금은 신체(의수, 의족, 의안, 의치 등 신체보조장구는 제외하나, 인공장기나 부분 의치 등 신체에 이식되어 그 기능을 대신할 경우는 포함합니다)에 입은 상해로 인한 보상금으로, 장해분류표에서 정한 기준에 따른 장해상태를 포함합니다. 또한, 장해분류표에서 정한 지급률을 보험가입금액에 곱하여 산출한 금액을 보험수익자에게 지급합니다. 보험금은 보험가입금액의 100%에 해당합니다. 보험금은 보험가'
# len(text)

In [12]:
# question = "접촉 사고 시 보장 사항"
# answer = query_with_rag(question, collection)
# output = answer.split('답변:')[-1]
# print(f'답변 : {output}')

In [13]:
# question = "점심 메뉴 알려줘"
# answer = query_with_rag(question, collection)
# output = answer.split('답변:')[-1]
# print(f'답변 : {output}')

In [14]:
# output = answer.split('답변:')[-1]

In [15]:
question = "운전자 보험 가입 후 계약을 철회할 수 있는 기간은 언제까지인가요?"
answer = query_with_rag(question, collection)
output = answer.split('답변:')[-1]
print(f'답변 : {output}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


답변 :  계약자에게 보험기간이 끝난다는 사실을 통보하고 재가입 의사를 문의 한 후, 계약자가 보험기간 종료일까지 제1항의 재가입 의사를 회사가 정한 방식으로 표시하면 별도의 청약 및 승낙의 절차를 거치지 않고 계약이 재가입 처리됩니다. 따라서 계약을 철회할 수 있는 기간은 보험기간 종료일까지 3개월 이내입니다. 이 기간 동안 계약을 철회하면 이미 납입한 보험료를 돌려드리며, 보험료를 받은 기간에 대하여 소정의 이자를 더하여 지급합니다. 따라서, 계약을 철회할 수 있는 기간은 보험기간 종료일까지 3개월 이내입니다. 이 기간 동안 계약을 철회하면 이미 납입한 보험료를 돌려드리며, 보험료를 받


In [16]:
question = "보험금 지급에 관한 세부규정에 대해서 알려줘"
answer = query_with_rag(question, collection)
output = answer.split('답변:')[-1]
print(f'답변 : {output}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


답변 :  보험금 지급에 관한 세부규정은 보험회사가 보험수익자에게 보험금을 지급할 때의 조건과 절차를 규정합니다. 보험금 지급에 관한 세부규정은 보험계약서 또는 보험회사에서 제공하는 서류와 계약서에 명시되어 있습니다. 보험금 지급에 관한 세부규정은 다음과 같은 사항을 포함합니다:
1. 지급 사유: 보험금 지급에 대한 사유를 명시합니다. 예를 들어, 보험금은 상해로 인한 사망, 장해, 입원 등에 의한 경우에 지급됩니다.
2. 지급 기준: 보험금 지급 기준을 명시합니다. 예를 들어, 보험금은 보험가입금액에 곱한 금액으로 지급됩니다.
3. 지급 절차: 보험금 지급 절


In [17]:
question = "피곤한데 잠 깨는 방법이 있을까?"
answer = query_with_rag(question, collection)
output = answer.split('답변:')[-1]
print(f'답변 : {output}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


답변 :  잠 깨는 방법은 여러 가지가 있지만, 주의해야 할 점은 잠 깨는 방법을 자주 사용하지 않도록 하는 것이 중요합니다. 다음은 잠 깨는 방법 중 몇 가지입니다:

1. **음악**: 조용한 음악을 듣는 것은 잠을 자는 데 도움이 될 수 있습니다. 하지만, 음악이 너무 재미있거나 흥분적인 경우 잠 깨는 경향이 있습니다.
2. **조명**: 조명이 적절하지 않으면 잠을 깨는 경향이 있습니다. 조명이 약한 상태에서 잠을 자는 것이 좋습니다.
3. **온도**: 온도가 적절하지 않으면 잠을 깨는 경향이 있습니다. 온도가 너무 높거나 낮을 경우 잠을 자는 것이 어려울 수 있습니다.
4. **운동**: 잠을 자는 동안 간단한 운동을 하는 것도 도움이 될 수 있습니다. 하지만, 너무 큰 운동은


In [18]:
question = "지금 눈이 오는데 집 가는 길이 많이 막힐까?"
answer = query_with_rag(question, collection)
output = answer.split('답변:')[-1]
print(f'답변 : {output}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


답변 :  눈이 오면 교통이 원활하지 않을 수 있지만, 눈이 오는 상황에서 교통사고가 발생할 가능성은 적을 것 같습니다. 교통사고가 발생할 가능성은 주로 운전자의 과도한 속도, 통행금지 또는 일시정지, 도로의 상태 등에 의해 발생합니다. 따라서, 주의ful한 운전과 교통 상황을 주의深く 확인하면 문제가 발생하지 않을 것입니다. 만약 눈이 오는 상황에서 교통사고가 발생한다면, 교통사고처리특례법 제3조 제2항 단서에 따라 발생한 사고를 확인하고, 필요한 경우 법적 조치를 취하는 것이 중요합니다. 

결론: 눈이 오는 상황에서는 교통이 원활하지 않을 수 있지만, 주의ful한 운전과 교통 상황을 주의深く 확인하면 문제가
